In [1]:
from datasets import (
    import_main_class,
    list_datasets,
    load_dataset,
    load_dataset_builder,
    prepare_module,
)
    
ds_list = list_datasets(with_details=True)
ds_list[0].id, ds_list[0]

('acronym_identification',
 datasets.ObjectInfo(
 	id='acronym_identification',
 	description='Acronym identification training and development sets for the acronym identification task at SDU@AAAI-21.',
 	files=None
 ))

In [10]:
# list of fiels we want to keep around from the DatasetInfo object
keep_info_fields = [
    "features",
    "config_name",
    "splits",
]

# get DatasetInfo object without downloading the dataset
def get_config_infos(name):
    module_path, *_ = prepare_module(name, dataset=True)
    builder_cls = import_main_class(module_path, dataset=True)
    configs = [c.name for c in builder_cls.BUILDER_CONFIGS] or [None]
    if len(configs) == 1:
        info_dict = load_dataset_builder(name).info.__dict__
        return [
            {k: info_dict[k] for k in keep_info_fields}
        ]
    else:
        config_list = []
        for config_name in configs:
            info_dict = load_dataset_builder(name, config_name).info.__dict__
            config_list += [
                {k: info_dict[k] for k in keep_info_fields}
            ]
        return config_list
    
# Recursively get a list of all features of a certain dtype
# the output is a list of tuples > e.g. ('A', 'B', 'C') for feature example['A']['B']['C']
def get_typed_features(features, ftype='string', parents=None):
    if parents is None:
        parents = []
    typed_features = []
    for name, feat in features.items():
        if hasattr(feat, 'dtype') and feat.dtype == ftype:
            typed_features += [tuple(parents + [name])]
        elif hasattr(feat, 'feature'):
            if hasattr(feat.feature, 'dtype') and feat.feature.dtype == ftype:
                typed_features += [tuple(parents + [name])]
            elif isinstance(feat.feature, dict):
                typed_features += get_typed_features(feat.feature, ftype, parents + [name])
    return typed_features

# Recursively get a list of all features that are ClassLabels
# the outputs are pairs of tuples as above and the list of class names
def get_label_features(features, parents=None):
    if parents is None:
        parents = []
    text_features = []
    for name, feat in features.items():
        if hasattr(feat, 'num_classes'):
            text_features += [(tuple(parents + [name]), feat.names)]
        elif hasattr(feat, 'feature'):
            if hasattr(feat.feature, 'num_classes'):
                text_features += [(tuple(parents + [name]), feat.feature.names)]
            elif isinstance(feat.feature, dict):
                text_features += get_label_features(feat.feature, parents + [name])
    return text_features

# Cast info to pure dictionary by casting SplitInfo and pre-selecting features
def dictionarize_info(info_dict):
    res = {}
    res["config_name"] = info_dict["config_name"]
    res["splits"] = {
        spl: spl_info.num_examples
        for spl, spl_info in info_dict["splits"].items()
    }
    res["features"] = {
        "string": get_typed_features(info_dict["features"], 'string'),
        "int32": get_typed_features(info_dict["features"], 'int32'),
        "float32": get_typed_features(info_dict["features"], 'float32'),
        "label": get_label_features(info_dict["features"]),
    }
    return res

# All together now!
def get_config_infos_dict(name):
    return {
        config_info["config_name"]: dictionarize_info(config_info)
        for config_info in get_config_infos(name)
    }

In [7]:
glue_configs = get_config_infos_dict('glue')
glue_configs

{'cola': {'config_name': 'cola',
  'splits': {'test': 1063, 'train': 8551, 'validation': 1043},
  'features': {'string': [('sentence',)],
   'int32': [('idx',)],
   'float32': [],
   'label': [(('label',), ['unacceptable', 'acceptable'])]}},
 'sst2': {'config_name': 'sst2',
  'splits': {'test': 1821, 'train': 67349, 'validation': 872},
  'features': {'string': [('sentence',)],
   'int32': [('idx',)],
   'float32': [],
   'label': [(('label',), ['negative', 'positive'])]}},
 'mrpc': {'config_name': 'mrpc',
  'splits': {'test': 1725, 'train': 3668, 'validation': 408},
  'features': {'string': [('sentence1',), ('sentence2',)],
   'int32': [('idx',)],
   'float32': [],
   'label': [(('label',), ['not_equivalent', 'equivalent'])]}},
 'qqp': {'config_name': 'qqp',
  'splits': {'train': 363846, 'validation': 40430, 'test': 390965},
  'features': {'string': [('question1',), ('question2',)],
   'int32': [('idx',)],
   'float32': [],
   'label': [(('label',), ['not_duplicate', 'duplicate'])]}},


In [ ]:
import tqdm

all_configs = []
for ds in tqdm.tqdm(ds_list[:500]):
    try:
        all_configs += [{
            "name": ds.id,
            "description": ds.description,
            "configs": get_config_infos_dict(ds.id),
        }]
    except:
        print(f"---- MISSED ---- {ds.id}")

name_to_configs = {x["name"]: x for x in all_configs}

 16%|█▌        | 78/500 [05:02<28:15,  4.02s/it]

---- MISSED ---- chr_en


 20%|██        | 100/500 [11:10<1:38:46, 14.82s/it]

 20%|██        | 101/500 [11:15<1:19:07, 11.90s/it]

 20%|██        | 102/500 [11:27<1:18:51, 11.89s/it]

...

 82%|████████▏ | 408/500 [1:06:35<3:11:13, 124.72s/it]

 82%|████████▏ | 409/500 [1:07:47<2:45:07, 108.87s/it]

 82%|████████▏ | 410/500 [1:07:56<1:58:31, 79.01s/it] 

 82%|████████▏ | 411/500 [1:08:00<1:23:44, 56.45s/it]

 82%|████████▏ | 412/500 [1:08:03<59:22, 40.48s/it]  

In [21]:
list(name_to_configs.keys())

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus',
 'ag_news',
 'ai2_arc',
 'air_dialogue',
 'ajgt_twitter_ar',
 'allegro_reviews',
 'allocine',
 'alt',
 'amazon_polarity',
 'amazon_reviews_multi',
 'amazon_us_reviews',
 'ambig_qa',
 'amttl',
 'anli',
 'app_reviews',
 'aqua_rat',
 'aquamuse',
 'ar_cov19',
 'ar_res_reviews',
 'ar_sarcasm',
 'arabic_billion_words',
 'arabic_pos_dialect',
 'arabic_speech_corpus',
 'arcd',
 'arsentd_lev',
 'art',
 'arxiv_dataset',
 'ascent_kb',
 'aslg_pc12',
 'asnq',
 'asset',
 'assin',
 'assin2',
 'atomic',
 'autshumato',
 'babi_qa',
 'banking77',
 'bbaw_egyptian',
 'bbc_hindi_nli',
 'bc2gm_corpus',
 'best2009',
 'bianet',
 'bible_para',
 'big_patent',
 'billsum',
 'bing_coronavirus_query_set',
 'biomrc',
 'blended_skill_talk',
 'blimp',
 'blog_authorship_corpus',
 'bn_hate_speech',
 'bookcorpus',
 'bookcorpusopen',
 'boolq',
 'bprec',
 'break_data',
 'brwac',
 'bsd_ja_en',
 'bswac',
 'c3',
 'c4',
 'cail2018

In [25]:
# extract text to analyze:
# list of all examples of a given feature in a given split of a given config of a given dataset
# returns a list of strings
def get_text_to_analyze(
    name, text_path,
    config_name=None, split=None,
    max_items=20000, streaming=False
):
    ### default arguments
    if config_name is None:
        config_name = list(name_to_configs[name]["configs"])[0]
        print(f"using default config: {config_name}")
    config = name_to_configs[name]["configs"][config_name]
    if split is None:
        split = 'train' if 'train' in config["splits"] else list(config["splits"])[0]
        print(f"using default split: {split}")        
    ### get text from dataset
    print(f"running -- load_dataset({name}, {config_name}, streaming={streaming})")
    dataset = load_dataset(name, config_name, streaming=streaming)
    text_list = []
    example_ct = 0
    for example in dataset[split]:
        example_ct += 1
        # robustly handle fields that contain lists of text
        item_list = [example]
        for field_name in text_path:
            item_list = [
                next_item
                for item in item_list
                for next_item in (item[field_name] if isinstance(item[field_name], list) else [item[field_name]])
            ]
        text_list += [
            text
            for item in item_list
            for text in (item if isinstance(item, list) else [item])
        ]
        if example_ct >= max_items:
            break
    return text_list

In [23]:
name_to_configs["ai2_arc"]

{'name': 'ai2_arc',
 'description': 'A new dataset of 7,787 genuine grade-school level, multiple-choice science questions, assembled to encourage research in\n advanced question-answering. The dataset is partitioned into a Challenge Set and an Easy Set, where the former contains\n only questions answered incorrectly by both a retrieval-based algorithm and a word co-occurrence algorithm. We are also\n including a corpus of over 14 million science sentences relevant to the task, and an implementation of three neural baseline models for this dataset. We pose ARC as a challenge to the community.',
 'configs': {'ARC-Challenge': {'config_name': 'ARC-Challenge',
   'splits': {'test': 1172, 'train': 1119, 'validation': 299},
   'features': {'string': [('id',),
     ('question',),
     ('choices', 'text'),
     ('choices', 'label'),
     ('answerKey',)],
    'int32': [],
    'float32': [],
    'label': []}},
  'ARC-Easy': {'config_name': 'ARC-Easy',
   'splits': {'test': 2376, 'train': 2251, 'v

In [26]:
get_text_to_analyze(
    "ai2_arc", ('choices', 'text'),
    config_name="ARC-Challenge", split="train",
    max_items=20, streaming=True
)

running -- load_dataset(ai2_arc, ARC-Challenge, streaming=True)


['dry palms',
 'wet palms',
 'palms covered with oil',
 'palms covered with lotion',
 'The refrigerator door is smooth.',
 'The refrigerator door contains iron.',
 'The refrigerator door is a good conductor.',
 'The refrigerator door has electric wires in it.',
 'cooling of flowing magma.',
 'converging of crustal plates.',
 'deposition of river sediments.',
 'solution of carbonate minerals.',
 'worldwide disease',
 'global mountain building',
 'rise of mammals that preyed upon plants and animals',
 'impact of an asteroid created dust that blocked the sunlight',
 'west',
 'east',
 'north',
 'south',
 'valleys carved by a moving glacier',
 'piles of rocks deposited by a melting glacier',
 'grooves created in a granite surface by a glacier',
 'bedrock hills roughened by the passing of a glacier',
 'Tissues in a single-celled organism are like the cells in a multi-celled organism.',
 'The nucleus in a single-celled organism is like the skin of a multi-celled organism.',
 'Organelles in a 

In [27]:
get_text_to_analyze(
    "ai2_arc", ('question',),
    config_name="ARC-Challenge", split="train",
    max_items=20, streaming=True
)

running -- load_dataset(ai2_arc, ARC-Challenge, streaming=True)


['George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?',
 'Which of the following statements best explains why magnets usually stick to a refrigerator door?',
 'A fold observed in layers of sedimentary rock most likely resulted from the',
 'Which of these do scientists offer as the most recent explanation as to why many plants and animals died out at the end of the Mesozoic era?',
 'A boat is acted on by a river current flowing north and by wind blowing on its sails. The boat travels northeast. In which direction is the wind most likely applying force to the sails of the boat?',
 'Which land form is the result of the constructive force of a glacier?',
 'Which statement best compares single-celled and multi-celled organisms?',
 'As part of an experiment, an astronaut takes a scale to the Moon and weighs himself. The scale reads 31 pounds. If the astronaut has a mass of about 84 kilograms, which are the approximate weight and mass of the 